<a href="https://colab.research.google.com/github/01star01ek/01star01ek/blob/main/%EC%A7%84%EC%A7%9C%EC%98%81%EC%83%81%EC%97%90%EC%84%9Cswimswap%EC%96%B4%EC%BC%80%EC%86%8D%E3%85%88%EB%8F%84%EC%A4%84%EC%97%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --force-reinstall "numpy<2.0"

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
%cd /content
!git clone -q https://github.com/woctezuma/SimSwap.git SimSwap

!pip install -q torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118
!pip install -q insightface==0.7.3 onnxruntime moviepy opencv-python imageio==2.34.0

/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.0.1+cu118 which is incompatible.


In [5]:
import cv2
import imageio
import subprocess
import os
from glob import glob

import sys
# ----------------------[2. 모델 다운로드 및 압축 해제]-----------------------
print("📥 모델 다운로드...")
!mkdir -p arcface_model parsing_model/checkpoint checkpoints insightface_func/models
!wget -q https://github.com/woctezuma/SimSwap-colab/releases/download/1.0/arcface_checkpoint.tar -O arcface_model/arcface_checkpoint.tar
!wget -q https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip -O checkpoints.zip
!wget -q https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth -O parsing_model/checkpoint/79999_iter.pth
!wget -q https://github.com/woctezuma/SimSwap-colab/releases/download/antelope/antelope.zip -O antelope.zip

!unzip -q checkpoints.zip -d checkpoints
!unzip -q antelope.zip -d insightface_func/models/

# SimSwap 설치 후 꼭 디렉토리 이동 필요
%cd /content/SimSwap

# ----------------------[3. PyTorch 호환성 및 감지 임계값 수정]-----------------------
!sed -i.bak 's/torch.load(netArc_checkpoint, map_location=torch.device("cpu"))/torch.load(netArc_checkpoint, map_location=torch.device("cpu"), weights_only=False)/' models/fs_model.py
!sed -i 's/det_size=(640,640)/det_size=(320,320)/' test_wholeimage_swapsingle.py

%cd /content/SimSwap
with open('test_wholeimage_swapsingle.py', 'r') as f:
    content = f.read()
if 'app.models["detection"].nms_thresh' not in content:
    new_lines = []
    for line in content.splitlines():
        new_lines.append(line)
        if 'app.prepare(' in line:
            new_lines.append('    if hasattr(app.models, "detection"):')
            new_lines.append('        app.models["detection"].nms_thresh = 0.3')
            new_lines.append('        app.models["detection"].det_thresh = 0.3')
    with open('test_wholeimage_swapsingle.py', 'w') as f:
        f.write('\n'.join(new_lines))

# ----------------------[4. 경로 및 환경 설정]-----------------------
source_img_path = '/content/input_photo/source.png'
target_video_path = '/content/input_video/target.mp4'
frame_dir = 'extracted_frames'
swap_dir = 'swapped_frames'
output_base_video = '/content/swapped_base_video.mp4'
output_final_video = '/content/swapped_interpolated_video.mp4'
frame_interval = 5
use_interpolation = True
arcface_path = '/content/arcface_model/arcface_checkpoint.tar'

os.makedirs(frame_dir, exist_ok=True)
os.makedirs(swap_dir, exist_ok=True)

# ----------------------[5. 개선된 프레임 추출 + FPS 계산]-----------------------
def extract_frames(video_path, frame_dir, interval=5):
    """개선된 프레임 추출 함수 - 진행상황 표시 및 정확한 FPS 계산"""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"비디오를 열 수 없습니다: {video_path}")
        return None, None

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    new_fps = original_fps / interval

    print(f"원본 FPS: {original_fps:.2f} → 새 FPS: {new_fps:.2f}")
    print(f"총 프레임: {total_frames}, 예상 저장 프레임: {total_frames // interval}개")

    frame_count = 0
    saved_count = 0

    print("프레임 추출 중...")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % interval == 0:
            cv2.imwrite(os.path.join(frame_dir, f'frame_{saved_count:04d}.jpg'), frame)
            saved_count += 1

            # 진행상황 표시 (50개마다)
            if saved_count % 50 == 0:
                progress_percent = (frame_count / total_frames) * 100
                print(f"추출 진행: {saved_count}개 프레임 저장됨... ({progress_percent:.1f}%)")

        frame_count += 1

    cap.release()
    print(f"✅ 프레임 추출 완료: {saved_count}장")
    return original_fps, new_fps

# 프레임 추출 실행
original_fps, new_fps = extract_frames(target_video_path, frame_dir, frame_interval)

if original_fps is None:
    print("❌ 프레임 추출 실패")
    import sys
    sys.exit()

# ----------------------[6. 개선된 SimSwap 얼굴 합성]-----------------------
%cd /content/SimSwap
def run_simswap_on_frames(source_img, frame_dir, swap_dir):
    """개선된 SimSwap 처리 함수 - 진행상황 표시 추가"""
    frames = sorted([f for f in os.listdir(frame_dir) if f.endswith('.jpg')])
    total_frames = len(frames)

    print(f"SimSwap 얼굴 합성 시작: {total_frames}개 프레임 처리")

    for i, frame_name in enumerate(frames):
        target_img = os.path.join(frame_dir, frame_name)
        result_subdir = os.path.join(swap_dir, f'frame_{i:04d}')
        os.makedirs(result_subdir, exist_ok=True)

        !python test_wholeimage_swapsingle.py \
            --crop_size 224 \
            --use_mask \
            --no_simswaplogo \
            --name people \
            --Arc_path "{arcface_path}" \
            --pic_a_path "{source_img}" \
            --pic_b_path "{target_img}" \
            --pic_specific_path "{target_img}" \
            --output_path "{result_subdir}" \
            --checkpoints_dir /content/SimSwap/checkpoints

        # 진행상황 표시 (10개마다)
        if (i + 1) % 10 == 0:
            progress_percent = ((i + 1) / total_frames) * 100
            print(f"SimSwap 진행: {i + 1}/{total_frames} ({progress_percent:.1f}%)")

    print("✅ SimSwap 합성 완료")

run_simswap_on_frames(source_img_path, frame_dir, swap_dir)

# ----------------------[7. 합성된 이미지들을 비디오로 변환 (계산된 FPS 사용)]-----------------------
def create_video_from_swapped_results(swap_dir, output_video, target_fps):
    """SimSwap 결과를 비디오로 변환하는 함수"""
    print(f"합성된 이미지들로 비디오 생성 중... (FPS: {target_fps:.2f})")

    # SimSwap 결과 이미지들 수집
    result_images = []
    swap_subdirs = sorted([d for d in os.listdir(swap_dir) if os.path.isdir(os.path.join(swap_dir, d))])

    for subdir in swap_subdirs:
        subdir_path = os.path.join(swap_dir, subdir)
        # SimSwap은 결과를 특정 이름으로 저장함 (보통 첫 번째 이미지를 찾음)
        result_files = [f for f in os.listdir(subdir_path) if f.endswith(('.jpg', '.png'))]
        if result_files:
            result_images.append(os.path.join(subdir_path, result_files[0]))

    if not result_images:
        print("❌ 합성된 이미지를 찾을 수 없습니다.")
        return False

    # 첫 번째 이미지로 비디오 설정 확인
    first_img = cv2.imread(result_images[0])
    if first_img is None:
        print("❌ 첫 번째 이미지를 읽을 수 없습니다.")
        return False

    height, width, _ = first_img.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, target_fps, (width, height))

    if not out.isOpened():
        print("❌ 비디오 라이터를 열 수 없습니다.")
        return False

    print(f"{len(result_images)}개의 이미지를 {target_fps:.2f}FPS로 처리 중...")
    for i, img_path in enumerate(result_images):
        img = cv2.imread(img_path)
        if img is not None:
            out.write(img)

            # 진행상황 표시 (20개마다)
            if (i + 1) % 20 == 0:
                progress_percent = ((i + 1) / len(result_images)) * 100
                print(f"비디오 생성 진행: {i + 1}/{len(result_images)} ({progress_percent:.1f}%)")

    out.release()
    print(f"✅ 기본 비디오 생성 완료: {output_video}")
    return True

# 기본 비디오 생성
base_video_success = create_video_from_swapped_results(swap_dir, output_base_video, new_fps)

# ----------------------[8. FFmpeg를 이용한 프레임 보간]-----------------------
def interpolate_with_ffmpeg(input_video, output_video, original_fps, frame_interval):
    """FFmpeg minterpolate 필터로 보간 수행 (원본 FPS에 맞게 자동 계산)"""
    base_fps = original_fps / frame_interval
    target_fps = original_fps  # 최종 FPS는 원본과 동일
    multiplier = int(target_fps / base_fps)

    print(f"🔄 보간 시작: {input_video} → {target_fps:.2f}fps (x{multiplier})")
    print("보간 처리 중... (시간이 오래 걸릴 수 있습니다)")

    cmd = [
        'ffmpeg', '-i', input_video,
        '-filter:v', f'minterpolate=fps={target_fps}:mi_mode=mci:mc_mode=aobmc:vsbmc=1',
        '-y', output_video
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode == 0:
        print(f"✅ 보간 완료: {output_video}")
        return True
    else:
        print(f"❌ 보간 실패: {result.stderr}")
        return False

# 보간 실행
if base_video_success and use_interpolation and os.path.exists(output_base_video):
    print(f"\n🔄 프레임 보간 시작...")
    interpolation_success = interpolate_with_ffmpeg(
        output_base_video,
        output_final_video,
        original_fps=original_fps,
        frame_interval=frame_interval
    )

    if interpolation_success:
        print(f"🎉 최종 보간 영상 완료: {output_final_video}")
    else:
        print("⚠️ 보간 실패, 기본 영상을 사용하세요.")

# ----------------------[9. 완료 안내]-----------------------
print("\n" + "="*50)
print("🎉 전체 처리 완료!")
print(f"📊 처리 정보:")
print(f"   • 원본 FPS: {original_fps:.2f}")
print(f"   • 프레임 간격: {frame_interval}")
print(f"   • 기본 영상 FPS: {new_fps:.2f}")
if use_interpolation:
    print(f"   • 보간 영상 FPS: {original_fps:.2f}")
print(f"\n📹 결과 파일:")
print(f"   ▶ 기본 영상: {output_base_video}")
if use_interpolation and os.path.exists(output_final_video):
    print(f"   ▶ 보간 영상: {output_final_video}")
print("="*50)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
multisepcific_dir: ./demo_file/multispecific
nThreads: 2
n_blocks_global: 6
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 3
n_local_enhancers: 1
name: people
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
no_simswaplogo: True
norm: batch
norm_G: spectralspadesyncbatch3x3
ntest: inf
onnx: None
output_nc: 3
output_path: swapped_frames/frame_0021
phase: test
pic_a_path: /content/input_photo/source.png
pic_b_path: extracted_frames/frame_0021.jpg
pic_specific_path: extracted_frames/frame_0021.jpg
resize_or_crop: scale_width
results_dir: ./results/
semantic_nc: 3
serial_batches: False
temp_path: ./temp_results
tf_log: False
use_dropout: False
use_encoded_image: False
use_mask: True
verbose: False
video_path: G:/swap_data/video/HSB_Demo_Trim.mp4
which_epoch: latest
-------------- End ----------------
(142, 366, 4)
/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_coll